In [29]:
#ライブラリのインポート
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.offline as py
import plotly.graph_objs as go
import cufflinks as cf
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import log_loss, accuracy_score, confusion_matrix

In [30]:
#データセットのインポート
df_project=pd.read_csv("../1_data/ks-projects-201801.csv")[["goal","pledged","state","backers"]]
#df_project=pd.read_csv("../1_data/ks-projects-201801.csv")[["goal","pledged","state","backers","country"]]

#表示
display(df_project.head(100))
df_project.describe()
df_project.info()

,goal,pledged,state,backers
0,1000.0,0.00,failed,0
1,30000.0,2421.00,failed,15
2,45000.0,220.00,failed,3
3,5000.0,1.00,failed,1
4,19500.0,1283.00,canceled,14
5,50000.0,52375.00,successful,224
6,1000.0,1205.00,successful,16
7,25000.0,453.00,failed,40
8,125000.0,8233.00,canceled,58
9,65000.0,6240.57,canceled,43


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378661 entries, 0 to 378660
Data columns (total 4 columns):
goal       378661 non-null float64
pledged    378661 non-null float64
state      378661 non-null object
backers    378661 non-null int64
dtypes: float64(2), int64(1), object(1)
memory usage: 11.6+ MB


In [31]:
#2値に分ける
df_project.loc[df_project['state'] == 'successful','state_n']=1
df_project.loc[df_project['state'] != 'successful','state_n']=0
display(df_project.head())

,goal,pledged,state,backers,state_n
0,1000.0,0.0,failed,0,0.0
1,30000.0,2421.0,failed,15,0.0
2,45000.0,220.0,failed,3,0.0
3,5000.0,1.0,failed,1,0.0
4,19500.0,1283.0,canceled,14,0.0


In [32]:
#stateを削除
drop_columns = ['state']
df_project = df_project.drop(drop_columns, axis = 1)

In [33]:
df_project.isnull().sum()

goal       0
pledged    0
backers    0
state_n    0
dtype: int64

In [34]:
#ダミー変数作成
#df_project=pd.get_dummies(df_project)
#display(df_project.head())

In [35]:
#相関マップ
df_project.corr().style.background_gradient()

,goal,pledged,backers,state_n
goal,1,0.00735789,0.00401195,-0.0243322
pledged,0.00735789,1,0.717079,0.111534
backers,0.00401195,0.717079,1,0.129109
state_n,-0.0243322,0.111534,0.129109,1


In [36]:
#ロジスティック回帰を実装
y = df_project["state_n"].values
X = df_project.drop('state_n', axis=1).values
print(X)
clf = SGDClassifier(loss='log', penalty='none', max_iter=10000, fit_intercept=True, random_state=1234, tol=1e-3)
clf.fit(X, y)

# 重みを取得して表示
w0 = clf.intercept_[0]
w1 = clf.coef_[0, 0]
w2 = clf.coef_[0, 1]
w3 = clf.coef_[0, 2]
print('w0 = {:.3f}, w1 = {:.3f}, w2 = {:.3f}, w3 = {:.3f}'.format(w0, w1, w2, w3))

[[1.000e+03 0.000e+00 0.000e+00]
 [3.000e+04 2.421e+03 1.500e+01]
 [4.500e+04 2.200e+02 3.000e+00]
 ...
 [1.500e+04 2.000e+01 1.000e+00]
 [1.500e+04 2.000e+02 6.000e+00]
 [2.000e+03 5.240e+02 1.700e+01]]
w0 = 683.130, w1 = -67290.567, w2 = 67188.535, w3 = 5616.228


In [37]:
# ラベルを予測
y_est = clf.predict(X)

# 対数尤度を表示
print('対数尤度 = {:.3f}'.format(- log_loss(y, y_est)))

# 正答率を表示
print('正答率 = {:.3f}%'.format(100 * accuracy_score(y, y_est)))

対数尤度 = -0.580
正答率 = 98.322%


In [39]:
# 予測値と正解のクロス集計
conf_mat = pd.DataFrame(confusion_matrix(y, y_est), 
                        index=['正解 = 失敗', '正解 = 成功'], 
                        columns=['予測 = 失敗', '予測 = 成功'])
conf_mat

,予測 = 失敗,予測 = 成功
正解 = 失敗,241765,2940
正解 = 成功,3415,130541
